# t-SNE 

## Introduction
Given a collection of N high-dimensional objects x1, x2, .. xN, how can we get a feel for how these objects are arranged in the data space?
t-SNE builds maps in which distances between points reflect the similarities in the data. 

Principal component analysis is typically used to reduce the dimensionality. However variance might not be a good reflection of similarity of the data. Thus enters the SNE. Previous techniques used include, isomap or locally linear embedding (preserving small pair-wise distances). 

t-SNE is an improvement over the locally linear embedding. 

## t-Distributed Stochastic Neighbor Embedding
Measure the pairwise similarities between the high-dimensional objects :
$$p_{ij} = \frac{exp(-||x_i - x_j||^2 / 2\sigma^2}{\sum_{k}{\sum_{l \neq k}{exp(-||x_k-x_l||^2 / 2\sigma^2}}}$$

In practice, we compute the conditional distribution instead of join distribution. We set the bandwidth $$\sigma_i$$ such that the conditional has a fixed perplexity

$$p_{i|j} = \frac{exp(-||x_i - x_j||^2 / 2\sigma^2}{\sum_{k}{\sum_{l \neq k}{exp(-||x_k-x_l||^2 / 2\sigma^2}}}$$

Finally, we symmetrize the conditionals 

$$p_{ij} = \frac{p_{i|j} + p_{j|i}}{2N}$$

### Lower dimension
In the lower dimensional space, we go with : 
$$q_{ij} = \frac{(1 + ||y_i - y_j||^2)^-1}{} $$

So here, $$ p_{ij} \text{should be similar to } q_{ij}$$. We measure the distance between these distributions using 
$$KL(P||Q) = \sum_{i}{\sum_{j \neq i}{p_{ij}log{\frac{p_{ij}}{q_{ij}}}}}$$

So basically, using a stochastic process, move the points around to minimize the KL measure above.

### Why use a t-distribution?
Why do we define lower dimensional map similarities with 
$$q_{ij} \propto (1 + ||y_i - y_j||^2)^{-1}$$
If we are embedding high dimensional data to lower dimensional, the dissimilar points need to be modelled far apart. If we have a guassian points with prob 0.1, for the same 0.1 in a t-distribution, we need to have the points too far apart. So we use a t-distribution. 

## Gradient Interpretation
We can interpret the t-SNE gradient as a simulation of an N-body system:
$$\frac{\partial{C}}{\partial{y_i}} = 4\sum_{i \neq j}{(p_{ij}-q_{ij})(1 + ||y_i - y_j||^2)^{-1}(y_i-y_j)}$$
This is like for point i, we take the sum of all the forces (error terms above) and find out how we can move the point, to minimize the force.
One major limitation is that we have to take all the n points and hence n^2 interactions which is tedious. We take an approximation of this computation which scales to nlogn. The intuition here is we take an approximation of the weighted interaction. 

### Implementation : Quadtree
In Quadtree (a space partitioning tree), each node has 4 children. Each of the nodes corresponds to the each quadrant and their children will be nodes. We proceed till we have each cell having one point. Say we want to find the interactions with F, then we do a Depth first search, and at each step, we ask if this cell is small enough / far enough to be used as summary for the interactions.So at the first level we find that the cell is far enough, we simply take the center of mass and multiply the average distance (just prune everything else under it). This gives a nlgn algo for the computation.

## Limitations of using a single map
If we are using the t-SNE for a non-metric data where the triangle inequality wouldn't be held, then the single map cannot be used for visualization. To get around this issue, a multiple maps can be used. 

### Multiple maps t-SNE
We define the similarities as : 
$$q_{ij} = \frac{\sum_{m}{\pi^{m}_k \pi^{m}_l}(1 + ||y_k^{m} - y_l^{m}||^2)^{-1}}{\sum_{m'}{\sum_{k \neq l}{\pi^{m'}_k \pi^{m'}_l}(1 + ||y_k^{m'} - y_l^{m'}||^2)^{-1}}}$$
Here $$\pi_i^m \text{ is the weight of word i in map m}$$. 